In [1]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
# from __future__ import print_function
from keras.utils import to_categorical
import os
from keras.preprocessing.image import ImageDataGenerator, load_img

Using TensorFlow backend.


In [11]:
from keras.applications import VGG16

#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
vgg_conv = Model(inputs=vgg_conv.input, outputs=vgg_conv.get_layer('fc1').output)

# Freeze all the layers
for layer in vgg_conv.layers[:]:
    layer.trainable = True

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000001FFBDCF7128> True
<keras.layers.convolutional.Conv2D object at 0x000001FFBDCF74A8> True
<keras.layers.convolutional.Conv2D object at 0x000001FFBF6D5DD8> True
<keras.layers.pooling.MaxPooling2D object at 0x000001FFBDCF7C50> True
<keras.layers.convolutional.Conv2D object at 0x000001FFBDCF7198> True
<keras.layers.convolutional.Conv2D object at 0x000001FFBDF16470> True
<keras.layers.pooling.MaxPooling2D object at 0x000001FFBDF34860> True
<keras.layers.convolutional.Conv2D object at 0x000001FFBDF4B978> True
<keras.layers.convolutional.Conv2D object at 0x000001FFBDF67EF0> True
<keras.layers.convolutional.Conv2D object at 0x000001FFBDF9C6D8> True
<keras.layers.pooling.MaxPooling2D object at 0x000001FFBDFB3B70> True
<keras.layers.convolutional.Conv2D object at 0x000001FFBDFB3F28> True
<keras.layers.convolutional.Conv2D object at 0x000001FFBDFED4E0> True
<keras.layers.convolutional.Conv2D object at 0x000001FFBE003E48> True
<keras.layers.pool

In [12]:
from keras import models
from keras import layers
from keras import optimizers

# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_4 (Model)              (None, 4096)              117479232 
_________________________________________________________________
dense_5 (Dense)              (None, 256)               1048832   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 771       
Total params: 118,528,835
Trainable params: 118,528,835
Non-trainable params: 0
_________________________________________________________________


In [14]:
train_dir = r"C:/Users/Hasan/Desktop/code/tutorial_deeplearning/clean-dataset/train" #r for raw string
validation_dir = r"C:/Users/Hasan/Desktop/code/tutorial_deeplearning/clean-dataset/validation"

In [15]:
# No Data augmentation 
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

# Change the batchsize according to your system RAM
train_batchsize = 32
val_batchsize = 10
inputSize = 224

# Data Generator for Training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(inputSize, inputSize),
        batch_size=train_batchsize,
        class_mode='categorical')

# Data Generator for Validation data
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(inputSize, inputSize),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)


Found 600 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [16]:
from keras import optimizers

model.compile(optimizer=optimizers.SGD(lr=1e-5, decay=1e-6,momentum=0.9,nesterov=True),
             loss = 'categorical_crossentropy', metrics=['accuracy'])

# Train the Model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Epoch 1/20
19/18 [==============================] - 867s 46s/step - loss: 4.4046 - acc: 0.4654 - val_loss: 0.7277 - val_acc: 0.7867
Epoch 2/20
19/18 [==============================] - 827s 44s/step - loss: 0.8814 - acc: 0.7834 - val_loss: 0.3648 - val_acc: 0.8867
Epoch 3/20
19/18 [==============================] - 811s 43s/step - loss: 0.4408 - acc: 0.8810 - val_loss: 0.2544 - val_acc: 0.9200
Epoch 4/20
 5/18 [=======>......................] - ETA: 9:14 - loss: 0.2835 - acc: 0.8875 

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(validation_features,validation_labels)
print(test_loss, test_acc)

In [ ]:
#Plot the Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
 
#Plot the Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)